# Learn the basics of Druid SQL

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->
  
Apache Druid supports two query languages: Druid SQL and native queries.
Druid SQL is a Structured Query Language (SQL) dialect that enables you to query datasources in Apache Druid using SQL statements.
SQL and Druid SQL use similar syntax, with some notable differences.
Not all SQL functions are supported in Druid SQL. Instead, Druid includes Druid-specific SQL functions for optimized query performance.

This interactive tutorial introduces you to the unique aspects of Druid SQL with the primary focus on the SELECT statement.
To learn about native queries, see [Native queries](https://druid.apache.org/docs/latest/querying/querying.html).

## Prerequisites

Make sure that you meet the requirements outlined in the README.md file of the [apache/druid repo](https://github.com/apache/druid/tree/master/examples/quickstart/jupyter-notebooks/).
Specifically, you need the following:
- Knowledge of SQL
- [Python3](https://www.python.org/downloads/)
- The [`requests` package](https://requests.readthedocs.io/en/latest/user/install/) for Python
- [JupyterLab](https://jupyter.org/install#jupyterlab) (recommended) or [Jupyter Notebook](https://jupyter.org/install#jupyter-notebook) running on a non-default port. Druid and Jupyter both default to port `8888`, so you need to start Jupyter on a different port. 
- An available Druid instance. This tutorial uses the automatic single-machine configuration described in the [Druid quickstart](https://druid.apache.org/docs/latest/tutorials/index.html), so no authentication or authorization is required unless explicitly mentioned. If you haven’t already, download Druid version 25.0 or higher and start Druid services as described in the quickstart.

This tutorial uses the [Druid Python API](Python_API_Tutorial.ipynb) to simplify access to Druid.

To start the tutorial, run the following cell. It imports the required Python packages and defines a variable for the Druid client, and another for the SQL client used to run SQL commands.

In [ ]:
import druidapi

# druid_host is the hostname and port for your Druid deployment. 
# In a distributed environment, you can point to other Druid services.
# In this tutorial, you'll use the Router service as the `druid_host`.
druid_host = "http://localhost:8888"
dataSourceName = "wikipedia-sql-tutorial"

druid = druidapi.jupyter_client(druid_host)
display = druid.display
sql_client = druid.sql

## Druid SQL statements

The following are the main Druid SQL statements:

* SELECT: extract data from a datasource
* INSERT INTO: create a new datasource or append to an existing datasource
* REPLACE INTO: create a new datasource or overwrite data in an existing datasource

Druid SQL does not support CREATE TABLE, DELETE, and DROP TABLE statements.

## Ingest data

You can use either INSERT INTO or REPLACE INTO to create a datasource and ingest data.
INSERT INTO and REPLACE INTO statements both require the PARTITIONED BY clause which defines the granularity of time-based partitioning. For more information, see [Partitioning by time](https://druid.apache.org/docs/latest/multi-stage-query/concepts.html#partitioning-by-time).

Run the following cell to ingest data from an external source into a table called `wikipedia-sql-tutorial`.  
If you already have a table with the same name, use REPLACE INTO instead of INSERT INTO.

Note the following about the query to ingest data:
- The query uses the TIME_PARSE function to parse ISO 8601 time strings into timestamps. See the section on [timestamp values](#timestamp-values) for more information.
- The asterisk ( * ) tells Druid to ingest all the columns.
- The EXTERN statement lets you define the data source type and the input schema. See [Read external data with EXTERN](https://druid.apache.org/docs/latest/multi-stage-query/concepts.html#read-external-data-with-extern) for more information.

The following cell defines the query, uses MSQ to ingest the data, and waits for the MSQ task to complete. You will see an asterisk `[*]` in the left margin while the task runs.

In [ ]:
sql = '''
INSERT INTO "wikipedia-sql-tutorial" 
SELECT TIME_PARSE("timestamp") AS __time, * 
FROM TABLE (EXTERN(
    '{"type": "http", "uris": ["https://druid.apache.org/data/wikipedia.json.gz"]}',
    '{"type": "json"}', 
    '[{"name": "added", "type": "long"}, {"name": "channel", "type": "string"}, {"name": "cityName", "type": "string"}, {"name": "comment", "type": "string"}, {"name": "commentLength", "type": "long"}, {"name": "countryIsoCode", "type": "string"}, {"name": "countryName", "type": "string"}, {"name": "deleted", "type": "long"}, {"name": "delta", "type": "long"}, {"name": "deltaBucket", "type": "string"}, {"name": "diffUrl", "type": "string"}, {"name": "flags", "type": "string"}, {"name": "isAnonymous", "type": "string"}, {"name": "isMinor", "type": "string"}, {"name": "isNew", "type": "string"}, {"name": "isRobot", "type": "string"}, {"name": "isUnpatrolled", "type": "string"}, {"name": "metroCode", "type": "string"}, {"name": "namespace", "type": "string"}, {"name": "page", "type": "string"}, {"name": "regionIsoCode", "type": "string"}, {"name": "regionName", "type": "string"}, {"name": "timestamp", "type": "string"}, {"name": "user", "type": "string"}]'
    ))
PARTITIONED BY DAY
'''
sql_client.run_task(sql)

MSQ reports task completion as soon as ingestion is done. However, it takes a while for Druid to load the resulting segments. Wait for the table to become ready.

In [ ]:
sql_client.wait_until_ready(dataSourceName)

## Datasources

Druid supports a variety of datasources, with the table datasource being the most common. In Druid documentation, the word "datasource" often implicitly refers to the table datasource.
The [Datasources](https://druid.apache.org/docs/latest/querying/datasource.html) topic provides a comprehensive overview of datasources supported by Druid SQL.

In Druid SQL, table datasources reside in the `druid` schema. This is the default schema, so table datasources can be referenced as either `druid.dataSourceName` or `dataSourceName`.

For example, run the next cell to return the rows of the column named `channel` from the `wikipedia-sql-tutorial` table. Because this tutorial is running in Jupyter, the cells use the LIMIT clause to limit the size of the query results for display purposes. The cell uses the built-in table formatting feature of the Python API. You can also retrieve the values as a Python object if you wish to perform additional processing.

In [ ]:
sql = '''
SELECT "channel" FROM "wikipedia-sql-tutorial" LIMIT 7
'''
display.sql(sql)

## Data types

Druid maps SQL data types onto native types at query runtime.
The following native types are supported for Druid columns:

* SQL: `VARCHAR`, Druid: `STRING`: UTF-8 encoded strings and string arrays
* SQL: `BIGINT`, Druid: `LONG`: 64-bit signed int
* SQL & Druid: `FLOAT`: 32-bit float
* SQL & Druid: `DOUBLE`: 64-bit float
* Druid `COMPLEX`: represents non-standard data types, such as nested JSON, hyperUnique and approxHistogram aggregators, and DataSketches aggregators

For reference on how SQL data types map onto Druid native types, see [Standard types](https://druid.apache.org/docs/latest/querying/sql-data-types.html#standard-types).

Druid exposes table and column metadata through [INFORMATION_SCHEMA](https://druid.apache.org/docs/latest/querying/sql-metadata-tables.html#information-schema) tables. Run the following query to retrieve metadata for the `wikipedia-sql-tutorial` datasource. In the response body, each JSON object correlates to a column in the table.
Check the objects' `DATA_TYPE` property for SQL data types. You should see TIMESTAMP, BIGINT, and VARCHAR SQL data types. 

In [ ]:
sql = '''
SELECT COLUMN_NAME, DATA_TYPE 
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE "TABLE_SCHEMA" = 'druid' AND "TABLE_NAME" = 'wikipedia-sql-tutorial' 
LIMIT 7
'''
display.sql(sql)

This is such a common query that the SQL client has it built in:

In [ ]:
display.table(dataSourceName)

### Timestamp values

Druid stores timestamp values as the number of milliseconds since the Unix epoch.
Primary timestamps are stored in a column named `__time`.
If a dataset doesn't have a timestamp, Druid uses the default value of `1970-01-01 00:00:00`.

Druid time functions perform best when used with the `__time` column.
By default, time functions use the UTC time zone.
For more information about timestamp handling, see [Date and time functions](https://druid.apache.org/docs/latest/querying/sql-scalar.html#date-and-time-functions).

Run the following cell to see a time function at work. This example uses the `TIME_IN_INTERVAL` function to query the `channel` and `page` columns of the `wikipedia-sql-tutorial` for rows whose timestamp is contained within the specified interval. The cell groups the results by columns.

In [ ]:
sql = '''
SELECT channel, page
FROM "wikipedia-sql-tutorial" 
WHERE TIME_IN_INTERVAL(__time, '2016-06-27T00:05:54.56/2016-06-27T00:06:53')
GROUP BY channel, page
LIMIT 7
'''
display.sql(sql)

### NULL values

Druid supports SQL compatible NULL handling, allowing string columns to distinguish empty strings from NULL and numeric columns to contain NULL rows. To store and query data in SQL compatible mode, explicitly set the `useDefaultValueForNull` property to `false` in `_common/common.runtime.properties`. See [Configuration reference](https://druid.apache.org/docs/latest/configuration/index.html) for common configuration properties.

When `useDefaultValueForNull` is set to `true` (default behavior), Druid stores NULL values as `0` for numeric columns and as `''` for string columns.

## SELECT statement syntax

Druid SQL supports SELECT statements with the following structure:

``` mysql
[ EXPLAIN PLAN FOR ]
[ WITH tableName [ ( column1, column2, ... ) ] AS ( query ) ]
SELECT [ ALL | DISTINCT ] { * | exprs }
FROM { <table> | (<subquery>) | <o1> [ INNER | LEFT ] JOIN <o2> ON condition }
[ WHERE expr ]
[ GROUP BY [ exprs | GROUPING SETS ( (exprs), ... ) | ROLLUP (exprs) | CUBE (exprs) ] ]
[ HAVING expr ]
[ ORDER BY expr [ ASC | DESC ], expr [ ASC | DESC ], ... ]
[ LIMIT limit ]
[ OFFSET offset ]
[ UNION ALL <another query> ]
```

As a general rule, use the LIMIT clause with `SELECT *` to limit the number of rows returned. 

## WHERE clause

Druid SQL uses the [SQL WHERE clause](https://druid.apache.org/docs/latest/querying/sql.html#where) of a SELECT statement to fetch data based on a particular condition.

In most cases, filtering your results by time using the WHERE clause improves query performance.
This is because Druid partitions data into time chunks and having a time range allows Druid to skip over unrelated data.
At ingestion time, you can further partition segments within a time chunk using the CLUSTERED BY clause to improve locality.
At query time, using the WHERE clause to filter on clustered dimensions can improve query performance.

Druid supports range filtering on columns that contain long millisecond values, with the boundaries specified as ISO 8601 time intervals. This is suitable for the `__time` column, long metric columns, and dimensions with values that can be parsed as long milliseconds.
    
For example, the following cell uses a comparison operator on the `__time` field to filter results from a certain time range.

In [ ]:
sql = '''
SELECT channel, page, comment 
FROM "wikipedia-sql-tutorial" 
WHERE __time >= TIMESTAMP '2015-09-12 23:33:55' 
  AND namespace = 'Main' 
LIMIT 7
'''
display.sql(sql)

### Comparison operators

Druid SQL supports the following comparison operators. You can use these operators in conjunction with the WHERE clause to compare expressions.

- equal to (=)
- greater than(>)
- less than (<)
- greater than or equal (>=)
- less than or equal (<=)
- not equal to( <>)

For example, the next cell returns the first seven records that match the following criteria:
- `cityName` is not an empty string
- `countryIsoCode` value equals to `US`

In [ ]:
sql = '''
SELECT channel, page, comment 
FROM "wikipedia-sql-tutorial" 
WHERE "cityName" <> '' AND "countryIsoCode" = 'US' 
LIMIT 7
'''
display.sql(sql)

### Logical operators

Druid's handling of logical operators is comparable to SQL with a few exceptions. For example, if an IN list contains NULL, the IN operator matches NULL values. This behavior is different from the SQL IN operator, which does not match NULL values. For a complete list of logical SQL operators supported by Druid SQL, see [Logical operators](https://druid.apache.org/docs/latest/querying/sql-operators.html#logical-operators).

## GROUP BY clause

Druid SQL uses the [SQL GROUP BY](https://druid.apache.org/docs/latest/querying/sql.html#group-by) clause to separate items into groups, where each group is composed of rows with identical values. 
The GROUP BY clause is often used with [aggregation functions](https://druid.apache.org/docs/latest/querying/sql-aggregations.html), such as COUNT or SUM, to produce summary values for each group.

For example, the following cell counts all of the entries separated by the field `channel`. The output is limited to seven rows and has two fields: `channel` and `counts`. For each unique value of `channel`, Druid aggregates all rows having that value, counts the number of entries in the group, and assigns the results to a field called `counts`.

In [ ]:
sql = '''
SELECT channel, COUNT(*) AS counts 
FROM "wikipedia-sql-tutorial" 
GROUP BY channel 
LIMIT 7
'''
display.sql(sql)

You can further define the groups by specifying multiple dimensions.
Druid SQL supports using numbers in GROUP BY and ORDER BY clauses to refer to column positions in the SELECT clause.
Similar to SQL, Druid SQL uses one-based indexing to reference elements in SQL statements.

For example, the next cell aggregates entries grouped by fields `cityName` and `countryName`.
The output has three fields: `cityName`, `countryName`, and `counts`. For each unique combination of `cityName` and `countryName`, Druid aggregates all rows and averages the entries in the group.
The output is limited to seven rows for display purposes.

In [ ]:
sql = '''
SELECT cityName, countryName, COUNT(*) AS counts 
FROM "wikipedia-sql-tutorial" 
GROUP BY cityName, countryName 
LIMIT 7
'''
display.sql(sql)

## Query types

Druid SQL is optimized for the following [native query](https://druid.apache.org/docs/latest/querying/querying.html) types:
- Scan
- Timeseries
- TopN
- GroupBy

Native queries are low-level JSON-based queries designed to be lightweight and complete very quickly.
Druid translates SQL statements into native queries using the [Apache Calcite](https://calcite.apache.org/) data management framework. The queries are then executed by the Druid cluster.

To get information about how a Druid SQL query is translated into a native query type, add [EXPLAIN PLAN FOR](https://druid.apache.org/docs/latest/querying/sql.html#explain-plan) to the beginning of the query.
Alternatively, you can set up [request logging](https://druid.apache.org/docs/latest/configuration/index.html#request-logging).

### Scan

The Scan native query type returns raw Druid rows in streaming mode.
Druid SQL uses the Scan query type for queries that do not aggregate&mdash;queries that do not have GROUP BY or DISTINCT clauses.

For example, run the next cell to scan the `wikipedia-sql-tutorial` table for comments from Mexico City. Calcite translates this Druid SQL query into the Scan native query type.

In [ ]:
sql = '''
SELECT comment AS "Entry" 
FROM "wikipedia-sql-tutorial" 
WHERE cityName = 'Mexico City' 
LIMIT 7
'''
display.sql(sql)

### TopN

The TopN native query type returns a sorted set of results for the values in a given dimension according to some criteria. TopN results are always computed in memory. In some cases, the TopN query type delivers approximate ranking and results. To prevent this, set the `useApproximateTopN` query context parameter to `false` when calling the [Druid SQL API](https://druid.apache.org/docs/latest/querying/sql-api.html). See [SQL query context](https://druid.apache.org/docs/latest/querying/sql-query-context.html) for more information.

Druid SQL uses TopN for queries that meet the following criteria:
- queries that GROUP BY a single expression
- queries that have ORDER BY and LIMIT clauses
- queries that do not contain HAVING
- queries that are not nested

For example, the next cell returns the channels based on the number of events for each one in ascending order. Calcite translates this Druid SQL query into the TopN native query type.

In [ ]:
sql = '''
SELECT channel, count(*) as "Number of events" 
FROM "wikipedia-sql-tutorial" 
GROUP BY channel 
ORDER BY "Number of events" ASC 
LIMIT 5
'''
display.sql(sql)

### Timeseries

The Timeseries native query type returns an array of JSON objects, where each object represents a value asked for by the Timeseries query.

Druid SQL uses Timeseries for queries that meet the following criteria:
- queries that GROUP BY `FLOOR(__time TO unit)` or `TIME_FLOOR(__time, period)`
- queries that do not contain other grouping expressions
- queries that do not contain HAVING
- queries that are not nested
- queries that either have no ORDER BY clause or an ORDER BY clause that orders by the same expression as present in GROUP BY

For example, Calcite translates the following Druid SQL query into the Timeseries native query type:

In [ ]:
sql = '''
SELECT 
 countryName AS "Country", 
 SUM(deleted) AS deleted, 
 SUM(added) AS added 
FROM "wikipedia-sql-tutorial"
WHERE countryName = 'France' 
GROUP BY countryName , FLOOR(__time TO HOUR) 
LIMIT 7
'''
display.sql(sql)

### GroupBy

The GroupBy native query type returns an array of JSON objects where each object represents a grouping asked for by the GroupBy query. GroupBy delivers exact results and rankings.

Druid SQL uses GroupBy for aggregations, including nested aggregation queries.

For example, Calcite translates the following Druid SQL query into the GroupBy native query type:

In [ ]:
sql = '''
SELECT 
  countryName AS "Country\", 
  countryIsoCode AS "ISO" 
FROM "wikipedia-sql-tutorial"
WHERE channel = '#es.wikipedia' 
GROUP BY countryName, countryIsoCode 
LIMIT 7
'''
display.sql(sql)

## Learn more

This tutorial covers the basics of Druid SQL. To learn more about querying datasets using Druid SQL, see the following topics:

- [Druid SQL overview](https://druid.apache.org/docs/latest/querying/sql.html) to learn about Druid SQL syntax.
- [SQL data types](https://druid.apache.org/docs/latest/querying/sql-data-types.html) for information on how SQL data types map to Druid SQL.
- [SQL query translation](https://druid.apache.org/docs/latest/querying/sql-translation.html) for best practices that help you minimize the impact of SQL translation.
- [Druid SQL operators](https://druid.apache.org/docs/latest/querying/sql-operators.html) for operators supported by Druid SQL.
- [SQL aggregation functions](https://druid.apache.org/docs/latest/querying/sql-aggregations.html) for reference on the aggregation functions supported by Druid SQL. 
- [Unsupported features](https://druid.apache.org/docs/latest/querying/sql-translation.html#unsupported-features) for a list of SQL features not supported by Druid SQL.
- [SQL keywords](https://calcite.apache.org/docs/reference.html#keywords) for a list of SQL keywords.